In [1]:
import asyncio
import zipfile
import io
import requests
import json
import pandas as pd
from dotenv import load_dotenv
import os
from typing import List
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.docstore.document import Document
from langchain.memory import ChatMessageHistory, ConversationBufferMemory
from langchain.document_loaders import DataFrameLoader
from langchain.vectorstores import Qdrant
from qdrant_client import QdrantClient

In [2]:
load_dotenv()

True

In [3]:
system_template = """
You are PharmAssistAI, an AI assistant for pharmacists and pharmacy students. Use the following pieces of context to answer the user's question.

If you don't know the answer, simply state that you don't have enough information to provide an answer. Do not attempt to make up an answer.

ALWAYS include a "SOURCES" section at the end of your response, referencing the specific documents from which you derived your answer. 

If the user greets you with a greeting like "Hi", "Hello", or "How are you", respond in a friendly manner.

Example response format:
<answer>
SOURCES: <document_references>

Begin!
----------------
{summaries}
"""

messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template("{question}"),
]
prompt = ChatPromptTemplate.from_messages(messages)
chain_type_kwargs = {"prompt": prompt}

In [4]:
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

QDRANT_API_KEY = os.environ.get("QDRANT_API_KEY")
QDRANT_CLUSTER_URL = os.environ.get("QDRANT_CLUSTER_URL")

qdrant_client = QdrantClient(url=QDRANT_CLUSTER_URL, api_key=QDRANT_API_KEY, timeout=60)

response = qdrant_client.get_collections()
collection_names = [collection.name for collection in response.collections]

if "fda_drugs" not in collection_names:
    print("Collection 'fda_drugs' is not present.")
    
    # Download and process the FDA drug data
    url = "https://download.open.fda.gov/drug/label/drug-label-0001-of-0012.json.zip"
    response = requests.get(url)
    zip_file = zipfile.ZipFile(io.BytesIO(response.content))
    json_file = zip_file.open(zip_file.namelist()[0])
    data = json.load(json_file)
    
    df = pd.json_normalize(data['results'])
    selected_drugs = df
    
    # Define metadata fields and text fields
    metadata_fields = ['openfda.brand_name', 'openfda.generic_name', 'openfda.manufacturer_name',
                       'openfda.product_type', 'openfda.route', 'openfda.substance_name',
                       'openfda.rxcui', 'openfda.spl_id', 'openfda.package_ndc']
    text_fields = ['description', 'indications_and_usage', 'contraindications',
                   'warnings', 'adverse_reactions', 'dosage_and_administration']
    
    selected_drugs[text_fields] = selected_drugs[text_fields].fillna('')
    selected_drugs['content'] = selected_drugs[text_fields].apply(lambda x: ' '.join(x.astype(str)), axis=1)
    
    loader = DataFrameLoader(selected_drugs, page_content_column='content')
    drug_docs = loader.load()
    
    for doc, row in zip(drug_docs, selected_drugs.to_dict(orient='records')):
        metadata = {}
        for field in metadata_fields:
            value = row.get(field)
            if isinstance(value, list):
                value = ', '.join(str(v) for v in value if pd.notna(v))
            elif pd.isna(value):
                value = 'Not Available'
            metadata[field] = value
        doc.metadata = metadata
    
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
    split_drug_docs = text_splitter.split_documents(drug_docs)
    
    qdrant_vectorstore = Qdrant.from_documents(
        split_drug_docs,
        embedding_model,
        url=QDRANT_CLUSTER_URL,
        api_key=QDRANT_API_KEY,
        collection_name="fda_drugs"
    )
else:
    print("Collection 'fda_drugs' is present.")
    qdrant_vectorstore = Qdrant.construct_instance(
        texts=[""],
        embedding=embedding_model,
        url=QDRANT_CLUSTER_URL,
        api_key=QDRANT_API_KEY,
        collection_name="fda_drugs"
    )

/Users/raj/miniconda3/envs/llmops-course/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


Collection 'fda_drugs' is present.


In [11]:
def generate_answer(query):
    message_history = ChatMessageHistory()
    memory = ConversationBufferMemory(
        memory_key="chat_history",
        output_key="answer",
        chat_memory=message_history,
        return_messages=True,
    )

    chain = ConversationalRetrievalChain.from_llm(
        ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, streaming=True),
        chain_type="stuff",
        retriever=qdrant_vectorstore.as_retriever(),
        memory=memory,
        return_source_documents=True,
    )


    res = chain.invoke(query)
    answer = res["answer"]
    source_documents = res["source_documents"]


    text_elements = []
    if source_documents:
        for source_idx, source_doc in enumerate(source_documents):
            source_name = f"source_{source_idx}"
            text_elements.append(
                (source_doc.page_content, source_name)
            )
        source_names = [text_el[1] for text_el in text_elements]



    return answer, text_elements

In [12]:
query = "What should I be careful of when taking Metformin?"
answer, text_elements = generate_answer(query)
print(answer)

When taking Metformin, you should be cautious about excessive alcohol intake, both acute and chronic, as alcohol can potentiate the effects of Metformin on lactate metabolism. Additionally, Metformin should be temporarily discontinued before any intravascular radiocontrast study or surgical procedure. Patients with clinical or laboratory evidence of hepatic disease should generally avoid Metformin due to the risk of lactic acidosis. Symptoms of lactic acidosis can be subtle and include malaise, myalgias, respiratory distress, increasing somnolence, nonspecific abdominal distress, hypothermia, hypotension, and resistant bradyarrhythmias. If any of these symptoms occur, it is important to notify your physician immediately.


In [13]:
from langsmith import Client
from langsmith.evaluation import evaluate

Creating a LangSmith dataset

In [10]:
client = Client()

dataset_name = "PharmAssistAI Evaluation Dataset"
dataset = client.create_dataset(dataset_name, description="Evaluation dataset for PharmAssistAI application.")

client.create_examples(
    inputs=[
        {"question": "What should I be careful of when taking Metformin?"},
        {"question": "What are the contraindications of Aspirin?"},
        {"question": "I have been prescribed Metformin and Januvia - anything I should be careful of?"},
        {"question": "How does Januvia work?"}
    ],
    dataset_id=dataset.id,
)

Creating a custom evaluator

In [14]:
import re
from typing import Any, Optional
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.evaluation import StringEvaluator

class PharmAssistEvaluator(StringEvaluator):
    """An LLM-based evaluator for PharmAssistAI answers."""

    def __init__(self):
        #llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
        llm = ChatOpenAI(model="gpt-4", temperature=0)

        template = """On a scale from 0 to 100, how relevant and informative is the following response to the input question:
        --------
        QUESTION: {input}
        --------
        ANSWER: {prediction}
        --------
        Reason step by step about why the score is appropriate, considering the following criteria:
        - Relevance: Is the answer directly relevant to the question asked?
        - Informativeness: Does the answer provide sufficient and accurate information to address the question?
        - Clarity: Is the answer clear, concise, and easy to understand?
        - Sources: Are relevant sources cited to support the answer?
        
        Then print the score at the end. At the end, repeat that score alone on a new line."""

        self.eval_chain = PromptTemplate.from_template(template) | llm

    @property
    def requires_input(self) -> bool:
        return True

    @property
    def requires_reference(self) -> bool:
        return False

    @property
    def evaluation_name(self) -> str:
        return "pharm_assist_score"

    def _evaluate_strings(
        self,
        prediction: str,
        input: Optional[str] = None,
        reference: Optional[str] = None,
        **kwargs: Any
    ) -> dict:
        evaluator_result = self.eval_chain.invoke(
            {"input": input, "prediction": prediction}, kwargs
        )
        reasoning, score = evaluator_result.content.split("\n", maxsplit=1)
        score = re.search(r"\d+", score).group(0)
        if score is not None:
            score = float(score.strip()) / 100.0
        return {"score": score, "reasoning": reasoning.strip()}

Initializing our evaluator config

In [15]:
from langchain.smith import RunEvalConfig, run_on_dataset

eval_config = RunEvalConfig(
    custom_evaluators=[PharmAssistEvaluator()],
    evaluators=[
        "criteria",
        RunEvalConfig.Criteria("harmfulness"),
        RunEvalConfig.Criteria(
            {
                "AI": "Does the response feel AI generated? "
                "Respond Y if they do, and N if they don't."
            }
        ),
    ],
)

 Evaluating our RAG pipeline

In [16]:
def evaluate_pharmassist(example):
    query = example
    answer, text_elements = generate_answer(query)
    return {"answer": answer}

In [17]:
evaluate_pharmassist('What are the contraindications of Aspirin?')

{'answer': 'The contraindications of Aspirin include:\n1. Known allergy to nonsteroidal anti-inflammatory drug products (NSAIDs)\n2. Syndrome of asthma, rhinitis, and nasal polyps\n3. Children or teenagers for viral infections, with or without fever (risk of Reye syndrome)\n4. Patients with hemophilia\n5. Patients with significant respiratory depression or acute/severe bronchial asthma\n6. Patients with suspected or known paralytic ileus\n\nAdditionally, patients who consume three or more alcoholic drinks daily should be counseled about the bleeding risks associated with chronic, heavy alcohol use while taking aspirin.'}

In [19]:
# Execute an evaluation run on a specific dataset using a pre-configured client
client.run_on_dataset(
    dataset_name=dataset_name,  # Name of the dataset to use for the evaluation
    llm_or_chain_factory=evaluate_pharmassist,  # The language model or processing chain to be used for answering queries
    evaluation=eval_config,  # Evaluation configuration as defined previously, includes custom and built-in evaluators
    verbose=True,  # Enables verbose output to provide detailed logs during the execution
    project_name="PharmAssistAI - Eval",  # A descriptive name for the project, useful for logging and tracking purposes
    project_metadata={"version": "1.0.0"},  # Additional metadata for the project, useful for version control
)

View the evaluation results for project 'PharmAssistAI - Eval' at:
https://smith.langchain.com/o/bbdaa341-a469-5436-ba9e-24733ea4fe6d/datasets/cff0fec8-c26e-475c-b75c-ff22cefee71e/compare?selectedSessions=581015b0-67d1-4d5d-963e-fbda14645810

View all tests for Dataset PharmAssistAI Evaluation Dataset at:
https://smith.langchain.com/o/bbdaa341-a469-5436-ba9e-24733ea4fe6d/datasets/cff0fec8-c26e-475c-b75c-ff22cefee71e
[------------------------------------------------->] 4/4

,feedback.helpfulness,feedback.harmfulness,feedback.AI,feedback.pharm_assist_score,error,execution_time,run_id
count,4.00,4.0,4.00,4.000000,0,4.000000,4
unique,NaN,NaN,NaN,NaN,0,NaN,4
top,NaN,NaN,NaN,NaN,NaN,NaN,2cf2ad0c-598b-4438-891c-e41e023531e3
freq,NaN,NaN,NaN,NaN,NaN,NaN,1
mean,0.75,0.0,0.25,0.687500,NaN,3.394023,NaN
std,0.50,0.0,0.50,0.306526,NaN,0.936101,NaN
min,0.00,0.0,0.00,0.250000,NaN,2.149370,NaN
25%,0.75,0.0,0.00,0.587500,NaN,2.949774,NaN
50%,1.00,0.0,0.00,0.800000,NaN,3.592796,NaN
75%,1.00,0.0,0.25,0.900000,NaN,4.037044,NaN


{'project_name': 'PharmAssistAI - Eval',
 'results': {'c8ac04bf-a675-4c3a-ad42-064d48c4ff2b': {'input': {'question': 'What should I be careful of when taking Metformin?'},
   'feedback': [EvaluationResult(key='helpfulness', score=1, value='Y', comment='The criterion for this task is the helpfulness of the submission. \n\nThe submission provides a detailed explanation of what to be careful of when taking Metformin. It mentions the risks associated with alcohol intake, the need to discontinue Metformin before certain procedures, and the potential dangers for patients with hepatic disease. It also describes the symptoms of lactic acidosis, a possible side effect of Metformin, and advises the user to contact their physician if they experience these symptoms. \n\nThe submission is therefore helpful, insightful, and appropriate. It provides useful information that can help someone taking Metformin to use the medication safely and effectively. \n\nBased on this analysis, the submission meets 